In [1]:
import geopandas as gpd

# gdf = gpd.read_parquet('outputs/satellogic-earthview-items.parquet')
gdf = gpd.read_parquet('outputs/satellogic-earthview-items-with-matches.parquet')
gdf.shape

(1000000, 7)

In [65]:
sample = gdf.iloc[123]
sample

json_path     data/json/zone=14N/region=386613_2891668/date=...
zone                                                        14N
region                                           386613_2891668
date                                        2022-11-12 00:00:00
geometry      POLYGON ((-100.130416800567 26.136431441465, -...
s2_matches    [{'id': 'S2A_MSIL2A_20221120T171621_N0400_R112...
s1_matches    [{'id': 'S1A_IW_GRDH_1SDV_20221122T005052_2022...
Name: 873503, dtype: object

In [66]:
from eotdl.access import find_sentinel_matches_by_centroid

centroid = sample.geometry.centroid
matches = find_sentinel_matches_by_centroid(centroid, sample.date, 30, 384, 384, 'sentinel-2-l2a')
# matches = sample.s2_matches
matches

[{'id': 'S2B_MSIL2A_20221125T171639_N0400_R112_T14RLP_20221125T193955',
  'properties': {'datetime': '2022-11-25T17:26:46Z', 'eo:cloud_cover': 75.0}},
 {'id': 'S2B_MSIL2A_20221125T171639_N0400_R112_T14RLQ_20221125T193955',
  'properties': {'datetime': '2022-11-25T17:26:31Z', 'eo:cloud_cover': 99.25}},
 {'id': 'S2A_MSIL2A_20221120T171621_N0400_R112_T14RLP_20221120T215053',
  'properties': {'datetime': '2022-11-20T17:26:47Z', 'eo:cloud_cover': 94.56}},
 {'id': 'S2A_MSIL2A_20221120T171621_N0400_R112_T14RLQ_20221120T215053',
  'properties': {'datetime': '2022-11-20T17:26:32Z', 'eo:cloud_cover': 99.95}},
 {'id': 'S2B_MSIL2A_20221115T171559_N0400_R112_T14RLP_20221115T194629',
  'properties': {'datetime': '2022-11-15T17:26:46Z', 'eo:cloud_cover': 0.73}},
 {'id': 'S2B_MSIL2A_20221115T171559_N0400_R112_T14RLQ_20221115T194629',
  'properties': {'datetime': '2022-11-15T17:26:31Z', 'eo:cloud_cover': 10.35}},
 {'id': 'S2A_MSIL2A_20221110T171541_N0400_R112_T14RLP_20221110T215153',
  'properties': {'

In [67]:
matches = sample.s2_matches
matches

array([{'id': 'S2A_MSIL2A_20221120T171621_N0400_R112_T14RLP_20221120T215053', 'properties': {'datetime': '2022-11-20T17:26:47Z', 'eo:cloud_cover': 94.56}},
       {'id': 'S2A_MSIL2A_20221120T171621_N0400_R112_T14RLQ_20221120T215053', 'properties': {'datetime': '2022-11-20T17:26:32Z', 'eo:cloud_cover': 99.95}},
       {'id': 'S2B_MSIL2A_20221115T171559_N0400_R112_T14RLP_20221115T194629', 'properties': {'datetime': '2022-11-15T17:26:46Z', 'eo:cloud_cover': 0.73}},
       {'id': 'S2B_MSIL2A_20221115T171559_N0400_R112_T14RLQ_20221115T194629', 'properties': {'datetime': '2022-11-15T17:26:31Z', 'eo:cloud_cover': 10.35}},
       {'id': 'S2A_MSIL2A_20221110T171541_N0400_R112_T14RLP_20221110T215153', 'properties': {'datetime': '2022-11-10T17:26:50Z', 'eo:cloud_cover': 10.3}},
       {'id': 'S2A_MSIL2A_20221110T171541_N0400_R112_T14RLQ_20221110T215153', 'properties': {'datetime': '2022-11-10T17:26:35Z', 'eo:cloud_cover': 0.01}},
       {'id': 'S2B_MSIL2A_20221105T171509_N0400_R112_T14RLP_2022110

In [68]:
from datetime import datetime

s2_matches_filtered = [r for r in matches if r['properties']['eo:cloud_cover'] <= 10]
closest_match = min(s2_matches_filtered, key=lambda x: abs(datetime.fromisoformat(x['properties']['datetime'].replace('Z','')) - sample.date))
s2_date = closest_match['properties']['datetime']
s2_date

'2022-11-10T17:26:35Z'

In [69]:
from eotdl.access import download_sentinel_imagery
from eotdl.tools import bbox_from_centroid

custom_bbox = bbox_from_centroid(x=centroid.y, y=centroid.x, pixel_size=10, width=384, height=384)
download_sentinel_imagery('outputs/kk', s2_date, custom_bbox, 'sentinel-2-l2a', 'kk')